# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

In [ ]:
# 구글 드라이브 마운트, Colab을 이용하지 않는다면 패스해도 됩니다.
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)
# drive.mount('/content/drive')

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [1]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

from sklearn.model_selection import train_test_split # train-validation-test set 나누는 라이브러리


In [2]:
# 시드를 고정합니다.
SEED = 1234
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

## 1. ensemble
* 

In [3]:
from scipy.stats import mode

# # csv 일기
df_model1 = pd.read_csv("pred19_eff_b5_aug94200_Stop5_dc2_focalloss_2024-08-10-141847.csv")
df_model2 = pd.read_csv("pred21_eff_b5_aug94200_classweights_cosA_Stop5_2024-08-11-001541.csv")
df_model3 = pd.read_csv("pred17_eff_b5_aug94200_classweights_Stop5_2024-08-09-112318.csv")
# df_model4 = pd.read_csv("pred15_eff_b5_aug94200_dc0_Stop52024-08-08-173626.csv")
df_model5 = pd.read_csv("pred12_1_eff_b4_aug62800_decay0001_eStop5.csv")

model1 = df_model1['target']
model2 = df_model2['target']
model3 = df_model3['target']
# model4 = df_model4['target']
model5 = df_model5['target']



In [4]:
len(model2)

3140

In [5]:
# 모든 예측을 합쳐서 배열로 만듦
predictions = np.array([model1, model2, model3, model5])

# 각 데이터 포인트에 대해 다수결 투표로 최종 예측을 결정
ensemble_pred = mode(predictions, axis=0).mode

# 결과 출력 또는 저장
print(ensemble_pred)
print(len(ensemble_pred))

[ 2 12  5 ...  8  0 12]
3140


# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [6]:
pred_df = pd.DataFrame(df_model1, columns=['ID', 'target'])
pred_df['target'] = ensemble_pred

In [7]:
pred_df

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,12
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,12
4,00901f504008d884.jpg,2
...,...,...
3135,ffb4b6f619fb60ea.jpg,6
3136,ffb54299b1ad4159.jpg,10
3137,ffc2c91dff8cf2c0.jpg,8
3138,ffc4e330a5353a2a.jpg,0


In [8]:
sample_submission_df = pd.read_csv("data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [9]:
from datetime import datetime

now = datetime.now()
pred_df.to_csv(f"pred22_ensemble4_{now.strftime('%Y-%m-%d-%H%M%S')}.csv", index=False)

In [10]:
pred_df.head(20)

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,12
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,12
4,00901f504008d884.jpg,2
5,009b22decbc7220c.jpg,15
6,00b33e0ee6d59427.jpg,0
7,00bbdcfbbdb3e131.jpg,8
8,00c03047e0fbef40.jpg,15
9,00c0dabb63ca7a16.jpg,11


In [16]:
pred_df.head(20)

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,12
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,12
4,00901f504008d884.jpg,2
5,009b22decbc7220c.jpg,15
6,00b33e0ee6d59427.jpg,0
7,00bbdcfbbdb3e131.jpg,8
8,00c03047e0fbef40.jpg,15
9,00c0dabb63ca7a16.jpg,11


In [21]:
len(pred_df)

3140